# Creating Statements for MASSIVE dataset
- https://aclanthology.org/2023.acl-long.235/
- https://huggingface.co/datasets/AmazonScience/massive

Train dataset
- Statement, Label: True / False

In [5]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [17]:
# a lot of these came from the evaluation script so there are some that are unnecessary
from datasets import load_dataset, get_dataset_config_names, Dataset
import random
import numpy as np
import torch
import torch.nn.functional as F
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from torch.utils.data import DataLoader
import pandas as pd
from datasets import Dataset
from sklearn.utils import resample
from sklearn.utils import resample
from copy import copy
from torch.utils.data import DataLoader
import argparse
from tqdm import tqdm

In [18]:
SEED = 42
NUM_PROC=5
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
CACHE=None

In [6]:
from datasets import load_dataset
dataset = load_dataset('AmazonScience/massive', 'en-US', split='train')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/11514 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2033 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2974 [00:00<?, ? examples/s]

In [10]:
print(dataset.features['scenario'])

ClassLabel(names=['social', 'transport', 'calendar', 'play', 'news', 'datetime', 'recommendation', 'email', 'iot', 'general', 'audio', 'lists', 'qa', 'cooking', 'takeaway', 'music', 'alarm', 'weather'], id=None)


In [15]:
print(dataset.features['scenario'].names)
scenarios = dataset.features['scenario'].names

['social', 'transport', 'calendar', 'play', 'news', 'datetime', 'recommendation', 'email', 'iot', 'general', 'audio', 'lists', 'qa', 'cooking', 'takeaway', 'music', 'alarm', 'weather']


In [16]:
import numpy as np

values = set(dataset['scenario'])
print(values)

print(len(scenarios))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17}
18


In [11]:
dataset[0]

{'id': '1',
 'locale': 'en-US',
 'partition': 'train',
 'scenario': 16,
 'intent': 48,
 'utt': 'wake me up at nine am on friday',
 'annot_utt': 'wake me up at [time : nine am] on [date : friday]',
 'worker_id': '1',
 'slot_method': {'slot': [], 'method': []},
 'judgments': {'worker_id': [],
  'intent_score': [],
  'slots_score': [],
  'grammar_score': [],
  'spelling_score': [],
  'language_identification': []}}

In [62]:
# Create statements

def fill_template(templates, values):
    temp = random.sample(templates,1)[0]
    for i in range(len(values)):
        #print(f"i: {i}, values: {values[i]}")
        temp = temp.replace("${"+str(i+1)+"}", values[i])
    return temp

def generate_eval(dataset_name, templates, split, label_column, choices, question=None, template_key=None, template_labels=None, generate_other=False, input_sentences=[]):
  """
    - dataset_name: Path name to HuggingFace repo
    - templates: 2D list of templates
    - split: list of dataset split we are going to use, i.e. ['split']
    - question: column label
    - choices: optional choices (choice1 vs choice2 that we embed)
    - template_key: 'question' for XCOPA, 'label' for XNLI
    - template_labels: ['cause', 'effect'] for XCOPA, [0, 1, 2] for XNLI # choose which template we are going to use out of list of lists
    - generate_other: needed for XNLI
    - input_sentences: needed for XStoryCloze
  """
  langs = get_dataset_config_names(dataset_name)
  # sanity check: remove any langs that have more than 2 letters (should only be 2 letter code)
  # langs = [lang for lang in langs if len(lang) == 2]

  data = {}
  for lang in langs:
    print(f"loading dataset lang: {lang}")
    data[lang] = load_dataset(dataset_name, lang, split=split, cache_dir=CACHE)

  langs = [lang for lang in langs if 'all' not in lang]
  col_names = copy(data[langs[0]][0].column_names)
  #col_names.remove(label_column)

  def create_statements_labels(example):
    template=""
    # XCOPA, XNLI
    if template_key:
      if template_labels:
        for idx, val in enumerate(template_labels):
          if example[template_key] == val:
            template = templates[idx]

    # XWinograd
    if not template:
      template=templates

    # Choose from templates given
    temp = random.choice(template)

    statements = []

    right_answer = example[label_column]

    #idx = random.choice(range(0, len(choices)))
    # choose between 1 and 0
    truth_val = random.choice(range(0, 2))

    if truth_val:
      idx = right_answer
    else:
      idx = random.choice([i for i in range(len(choices)) if i != right_answer])

    example['label'] = 1 if idx == right_answer else 0
    values = []
    if question:
      values.append(example[question])
    values.append(choices[idx])
    example['statement'] = fill_template([temp], values)

    return example

  resulting_statements = {}
  for lang in langs:
    print(f"Processing {lang}...")
    resulting_statements[lang] = [split.map(create_statements_labels, remove_columns=col_names, num_proc=NUM_PROC) for split in data[lang]][0]

  return resulting_statements, langs

In [63]:
def create_dataloaders(statements, langs):
  dataloader_dict = {}
  for lang in langs:
    dataloader_dict[lang] = DataLoader(statements[lang], batch_size=32, shuffle=False)
  return dataloader_dict

In [64]:
# MASSIVE
print("Processing MASSIVE...")
dataset = "AmazonScience/massive"
templates = ["The utterance \"${1}\" is under the \"${2}\" scenario.", "Utterance: \"${1}\" Scenario: \"${2}\"", "User: \"${1}\". The best scenario for the user query is \"${2}\".", "The scenario of user's utterance \"${1}\" is \"${2}\"."]
split = ['train']
label_column = 'scenario'
question='utt'
choices=scenarios

massive_statements, massive_langs = generate_eval(dataset, templates, split, label_column, choices, question)

Processing MASSIVE...


/usr/local/lib/python3.10/dist-packages/datasets/load.py:1491: FutureWarning: The repository for AmazonScience/massive contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/AmazonScience/massive
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


loading dataset lang: af-ZA
loading dataset lang: am-ET
loading dataset lang: ar-SA
loading dataset lang: az-AZ
loading dataset lang: bn-BD
loading dataset lang: ca-ES
loading dataset lang: cy-GB
loading dataset lang: da-DK
loading dataset lang: de-DE
loading dataset lang: el-GR
loading dataset lang: en-US
loading dataset lang: es-ES
loading dataset lang: fa-IR
loading dataset lang: fi-FI
loading dataset lang: fr-FR
loading dataset lang: he-IL
loading dataset lang: hi-IN
loading dataset lang: hu-HU
loading dataset lang: hy-AM
loading dataset lang: id-ID
loading dataset lang: is-IS
loading dataset lang: it-IT
loading dataset lang: ja-JP
loading dataset lang: jv-ID
loading dataset lang: ka-GE
loading dataset lang: km-KH
loading dataset lang: kn-IN
loading dataset lang: ko-KR
loading dataset lang: lv-LV
loading dataset lang: ml-IN
loading dataset lang: mn-MN
loading dataset lang: ms-MY
loading dataset lang: my-MM
loading dataset lang: nb-NO
loading dataset lang: nl-NL
loading dataset lang

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

...

Map (num_proc=5):   0%|          | 0/11514 [00:00<?, ? examples/s]

Processing zh-CN...


Map (num_proc=5):   0%|          | 0/11514 [00:00<?, ? examples/s]

Processing zh-TW...


Map (num_proc=5):   0%|          | 0/11514 [00:00<?, ? examples/s]

In [65]:
values = massive_statements['en-US']['label']
unique, counts = np.unique(values, return_counts=True)

dict(zip(unique, counts)) # roughly 1 to 1

{0: 5783, 1: 5731}

In [66]:
print(massive_statements['en-US'][0])

{'label': 0, 'statement': 'The scenario of user\'s utterance "wake me up at nine am on friday" is "iot".'}


In [67]:
print(massive_statements['ko-KR'][0])

{'label': 0, 'statement': 'The scenario of user\'s utterance "금요일 오전 아홉 시 에 깨워줘" is "qa".'}


In [69]:
print(massive_statements['ko-KR'][406])

{'label': 1, 'statement': 'The scenario of user\'s utterance "한전 관련 뉴스 있으면 알려줘" is "news".'}


In [73]:
print(massive_statements['es-ES'][0])

{'label': 0, 'statement': 'User: "despiértame a las nueve de la mañana el viernes". The best scenario for the user query is "weather".'}


In [71]:
def push_dataset(statements, langs, dataset_name):
  for lang_code in langs:
    statements[lang_code].push_to_hub(f"mbzuai-ugrip-statement-tuning/{dataset_name}", lang_code[:2], split='train')

In [43]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [45]:
massive_statements['en-US'][0]

{'scenario': 16,
 'statement1': 'Utterance: "social" Scenario: "${2}"',
 'statement2': 'Utterance: "transport" Scenario: "${2}"',
 'statement3': 'Utterance: "calendar" Scenario: "${2}"',
 'statement4': 'Utterance: "play" Scenario: "${2}"',
 'statement5': 'Utterance: "news" Scenario: "${2}"',
 'statement6': 'Utterance: "datetime" Scenario: "${2}"',
 'statement7': 'Utterance: "recommendation" Scenario: "${2}"',
 'statement8': 'Utterance: "email" Scenario: "${2}"',
 'statement9': 'Utterance: "iot" Scenario: "${2}"',
 'statement10': 'Utterance: "general" Scenario: "${2}"',
 'statement11': 'Utterance: "audio" Scenario: "${2}"',
 'statement12': 'Utterance: "lists" Scenario: "${2}"',
 'statement13': 'Utterance: "qa" Scenario: "${2}"',
 'statement14': 'Utterance: "cooking" Scenario: "${2}"',
 'statement15': 'Utterance: "takeaway" Scenario: "${2}"',
 'statement16': 'Utterance: "music" Scenario: "${2}"',
 'statement17': 'Utterance: "alarm" Scenario: "${2}"',
 'statement18': 'Utterance: "weather"

In [72]:
push_dataset(massive_statements, massive_langs, "massive")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/12 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/71.8k [00:00<?, ?B/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/12 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/12 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/16.0k [00:00<?, ?B/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/12 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/14.9k [00:00<?, ?B/s]